In [1]:
import sys 
sys.path.append("/Users/germanignatovich/Desktop/projects/hse/TS/ts-project-assignment-hse-2023/functions")

In [2]:
import pandas as pd
from datetime import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
import yfinance as yf
from functions.data import get_NIC_from_csv

# Ticker specification

In [3]:
#ticker = yf.Ticker("NIC.AX")

#ticker.history(period='max', interval="1mo").to_csv('NICStockPriceMonthly.csv')
#cat.actions.to_csv('CaterpillatActions.csv')

# Market data import and transformation

In [4]:
# df = pd.read_csv('NICStockPriceMonthly.csv')
# df['Date'] = [pd.Timestamp(t, tz='Australia/Sydney') for t in df['Date']]
# #df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, utc=True)
# df.set_index('Date', inplace=True)
# df.drop(columns=['Open', 'High', 'Low'], inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: 'NICStockPriceMonthly.csv'

In [ ]:
print(df.shape)
df.head(3)

In [ ]:
date_cutoff = '2010-01-01'
df_m = df.asfreq(freq='m', method='ffill').fillna(method='ffill')[['Close', 'Volume']][date_cutoff:]
df_m['Year_Month'] = df_m.index.strftime('%Y-%m')

In [ ]:
actions = df[['Dividends', 'Stock Splits']]
actions = actions[date_cutoff:]
actions['Year_Month'] = actions.index.strftime('%Y-%m')
actions = actions.groupby('Year_Month', as_index=False).sum()

In [ ]:
data = df_m.merge(actions, how='inner', on='Year_Month')
data = data.set_index(df_m.index).drop(columns='Year_Month')

# Nickel Prices

In [ ]:
# import requests

# r = requests.get('https://markets.businessinsider.com/Ajax/Chart_GetChartData?instrumentType=Commodity&tkData=300002,10,0,333&from=20170817&to=20230309')
# nickelHist = pd.DataFrame(r.json())
# nickelHist['Date'] = [pd.Timestamp(t, tz='Australia/Sydney') for t in nickelHist['Date']]
# nickelHist.index = nickelHist['Date']
# nickelHist = nickelHist[nickelHist.index >= data.index[0]]
# nickelHist = nickelHist.asfreq(freq='m', method='bfill').fillna(method='bfill')
# data = data[data.index <= nickelHist.index[-1]]
# nickelHist.drop(columns=['Open', 'High', 'Low', 'Estimate', 'Volume', 'Date'], inplace=True)
# nickelHist.head(3)

In [ ]:
#nickelHist.to_csv('NickelPrices.csv')
Ni_prices = pd.read_csv('NickelPrices.csv')
Ni_prices['Date'] = [pd.Timestamp(t, tz='Australia/Sydney') for t in Ni_prices['Date']]
#Ni_prices['Date'] = pd.to_datetime(Ni_prices['Date'], infer_datetime_format=True, utc=True)
Ni_prices.set_index('Date', inplace=True)
data['Ni_price'] = Ni_prices['Close'].values

# Interest Rates Australia

In [ ]:
rates = pd.DataFrame(data=[
{'date': "03/08/2023",	'rate': 3.60},
{'date': "02/08/2023",	'rate': 3.35},
{'date': "12/07/2022",	'rate': 3.10},
{'date': "11/02/2022",	'rate': 2.85},
{'date': "10/05/2022",	'rate': 2.60},
{'date': "09/07/2022",	'rate': 2.35},
{'date': "08/03/2022",	'rate': 1.85},
{'date': "07/06/2022",	'rate': 1.35},
{'date': "06/08/2022",	'rate': 0.85},
{'date': "05/04/2022",	'rate': 0.35},
{'date': "11/04/2020",	'rate': 0.10},
{'date': "03/20/2020",	'rate': 0.25},
{'date': "03/04/2020",	'rate': 0.50},
{'date': "10/02/2019",	'rate': 0.75},
{'date': "07/03/2019",	'rate': 1.00},
{'date': "06/05/2019",	'rate': 1.25},
{'date': "08/03/2016",	'rate': 1.50}
] )

rates['date'] = [pd.Timestamp(t, tz='Australia/Sydney') for t in rates['date']]
rates.index = rates['date']
rates = rates.drop(columns=['date'])
rates = rates.asfreq(freq='m', method='bfill').fillna(method='bfill')
rates = rates[rates.index >= data.index[0]]

In [ ]:
data['aus_int_rate'] = rates['rate'].values

In [ ]:
data.head(2)

# Indonesia exports volume data
source: https://www.bi.go.id/en/statistik/ekonomi-keuangan/seki/Default.aspx#headingFour

In [ ]:
# import file
ind_exp = pd.read_excel("Indonesia_data_export/TABEL5_14_volume by sector.xls", sheet_name='5.14', header=4)

In [ ]:
# Transpose and reset index for later manipulations
ind_exp = ind_exp.transpose().reset_index()

In [ ]:
# Set column names
ind_exp_cols = ['Year', 'Month'] + list(ind_exp.iloc[176][2:69]) + ['drop1', 'drop2']
ind_exp.columns = ind_exp_cols

In [ ]:
# Drop empty rows
ind_exp = ind_exp[3:174].drop(columns=['drop1', 'drop2'])

In [ ]:
# indicies from 'Year' column with year as 2010, 2011 etc
bo = ~ind_exp['Year'].str.contains('Unnamed').fillna(False)
ind_exp['Year'] = ind_exp['Year'].where(bo).fillna(method='ffill')

In [ ]:
# Drop columns with yearly numbers
ind_exp = ind_exp[~ind_exp['Month'].isna()]

In [ ]:
# Remove excess characters
ind_exp['Month'] = ind_exp['Month'].str.replace('*', '')

In [ ]:
# Replace format of the month
ind_exp['Day'] = 1
months = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
ind_exp.replace({'Month': months}, inplace=True)

In [ ]:
# Create Date column
dt = pd.to_datetime(ind_exp[['Year', 'Month', 'Day']], format="%Y%m%d")
ind_exp['Date'] = [pd.Timestamp(t, tz='Australia/Sydney') for t in dt]


In [ ]:
# Set Date as Index and resample by last days of month
ind_exp.set_index('Date', inplace=True)
ind_exp = ind_exp.asfreq(freq='m', method='ffill').fillna(method='ffill')

In [ ]:
# Choose 'Base metal products' and 'Nickel ore'
ind_exp = ind_exp[['Base metal products', 'Nickel ore']]

# Slice period 
ind_exp = ind_exp['2018-09-30':]

In [ ]:
data[['Base_metal_products_vol', 'Nickel_ore_vol']] = ind_exp[['Base metal products', 'Nickel ore']]

# Indonesia exports value data
source: https://www.bi.go.id/en/statistik/ekonomi-keuangan/seki/Default.aspx#headingFour

In [ ]:
data